In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import  expected_conditions as ec
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains as AC

import time

import pandas as pd
import re

In [42]:
driver = webdriver.Chrome('chromedriver.exe')
url = ''
driver.get(url)

# r = re.compile('[^0-9a-zA-Z가-힣!?\s]')

body = driver.find_element_by_tag_name('body')

for i in range(10):
    time.sleep(1)
    body.send_keys(Keys.PAGE_DOWN)
    
wait = WebDriverWait(driver, 30)

coupang_reviews = pd.DataFrame(columns = {'score', 'comment', 'survey'})
now_page = 1

while True:
    time.sleep(3)
    wait.until(ec.presence_of_element_located((By.CLASS_NAME, 'js_reviewArticleListContainer')))
    wait.until(ec.presence_of_element_located((By.CLASS_NAME, 'sdp-review__article__page.js_reviewArticlePagingContainer')))
    list_con = driver.find_element_by_class_name('js_reviewArticleListContainer')
    review_list = list_con.find_elements_by_class_name('sdp-review__article__list.js_reviewArticleReviewList')

    for review in review_list:
        list_info = review.find_element_by_class_name('sdp-review__article__list__info')
        product_info = list_info.find_element_by_class_name('sdp-review__article__list__info__product-info')
        star_gray = product_info.find_element_by_class_name('sdp-review__article__list__info__product-info__star-gray')
        star_oran = star_gray.find_element_by_class_name(
            'sdp-review__article__list__info__product-info__star-orange.js_reviewArticleRatingValue')
        star = star_oran.get_attribute('data-rating')

        try:
            head = review.find_element_by_class_name('sdp-review__article__list__headline').text
            survey = review.find_element_by_class_name('sdp-review__article__list__survey').text

        except:
            head = ''
            survey = ''

        comment = head + " "

        try:
            comment_con = review.find_element_by_class_name('sdp-review__article__list__review.js_reviewArticleContentContainer')
            comment += comment_con.text

        except:
            pass
        
#         comment = r.sub('', comment)
        coupang_reviews = coupang_reviews.append({'score' : star, 'comment' : comment, 'survey' : survey}, ignore_index=True)

    page_con = list_con.find_element_by_class_name('sdp-review__article__page.js_reviewArticlePagingContainer')
    page_btn = page_con.find_elements_by_css_selector('button')
    last_page = int(page_btn[-2].get_attribute('data-page'))

    if(last_page == now_page):
        last_btn = page_btn[-1].is_enabled()

        if(last_btn == False):
            break

        else:
            now_page += 1
            next_btn = page_btn[-1]
            AC(driver).move_to_element(next_btn).click(next_btn).perform()

    else:
        now_page += 1
        next_page = now_page % 10

        if next_page == 0:
                next_page = 10

        next_btn = page_btn[next_page]
        staleElement = True
        AC(driver).move_to_element(next_btn).click(next_btn).perform()

driver.close()

In [43]:
coupang_reviews.to_excel('', encoding = 'euc-kr')